In [1]:
import spacy
import json
import re

C:\Users\Richy\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
nlp = spacy.load("fr_core_news_lg")

In [20]:
########################
######## TEST ##########
########################

#####Traitement NLP#####
text = "Moi, Jean-Pierre Dutroux, fils de Robert, je suis le président de la France (Université de Paris-Descartes)"
doc = nlp(text)
print("Entities before adding new entity:", doc.ents)

entities = []
for ent in doc.ents:
    entities.append((ent.text, ent.label_))

print(entities,"\n")

#####Conversion vers BIO#####
string_bio = ""
for entity in entities:
    chunk = entity[0].split(" ")
    #print(chunk)
    
    if len(chunk) > 1:
        for i in range(len(chunk)):
            if i == 0:
                string_bio += chunk[0]+" B-"+entity[1]+"\n"
            else:
                string_bio += chunk[i]+" I-"+entity[1]+"\n"
    else:
        string_bio += entity[0]+" O-"+entity[1]+"\n"

print(string_bio)

Entities before adding new entity: (Jean-Pierre Dutroux, Robert, Université de Paris-Descartes)
[('Jean-Pierre Dutroux', 'PER'), ('Robert', 'PER'), ('Université de Paris-Descartes', 'ORG')] 

Jean-Pierre B-PER
Dutroux I-PER
Robert O-PER
Université B-ORG
de I-ORG
Paris-Descartes I-ORG



In [3]:
def lire_csv(chemin):
    with open(chemin, "r") as f:
        dic = {}
        head = f.readline().split(";")
        
        for label in head:
            dic[label] = []
            
        for ligne in (f.readlines()):
            entity, n = ligne.split(";")
            entity = re.sub("[\"]","",entity)[2:]
            n = int(n[:-1])
            
            for label in head:
                for n in range(n):
                    dic[label].append([entity,"LOC"])
            
    return dic

def lire_fichier(chemin):
    with open(chemin,"r",encoding="utf-8") as f:
        return f.read()
    
def lire_json(chemin):
    with open(chemin,"r",encoding="utf-8") as f:
        return json.load(f)
    
def to_bio(liste_x, liste_y):
    string_X = ""
    for i in liste_x:
        mot = re.sub("\n","",i[0])
        chunk = re.split("[ -]", mot)
        for n in range(len(chunk)):
            if chunk[n] != " ":
                if n == 0:
                    string_X += chunk[0]+" B-"+i[1]+"\n"
                else:
                    string_X += chunk[n]+" I-"+i[1]+"\n"

    string_y = ""
    for i in liste_y:
        mot = re.sub("\n","",i[0])
        chunk = re.split("[ -;,:]", mot)
        for n in range(len(chunk)):
            if chunk[n]:
                if n == 0:
                    string_y += chunk[0]+" B-"+i[1]+"\n"
                else:
                    string_y += chunk[n]+" I-"+i[1]+"\n"
            
    return string_X, string_y

def to_bio_LOC(liste_x, liste_y):
    string_X = ""
    for i in liste_x:
        if i[1]:
            mot = re.sub("\n","",i[0])
            chunk = re.split("[ -]", mot)
            for n in range(len(chunk)):
                if chunk[n] != " ":
                    if n == 0:
                        string_X += chunk[0]+" B-"+i[1]+"\n"
                    else:
                        string_X += chunk[n]+" I-"+i[1]+"\n"

    string_y = ""
    for i in liste_y:
        if i[1]:
            mot = re.sub("\n","",i[0])
            chunk = re.split("[ -;,:]", mot)
            for n in range(len(chunk)):
                if chunk[n]:
                    if n == 0:
                        string_y += chunk[0]+" B-"+i[1]+"\n"
                    else:
                        string_y += chunk[n]+" I-"+i[1]+"\n"
            
    return string_X, string_y

def BIO_regroup(chemin):
    texte = lire_fichier(chemin)
    return re.sub(r"^  O (.*?)$", "\1\n\n", texte)
    
    #à faire: regroupement de lignes (pour avoir des tokens)

In [18]:
#######TESt sur les .csv#######

dic_x = lire_csv("maupassant_iter/MAUPASSANT_une-vie_PPtxt_lg_spacy_iterations.csv")
dic_y = lire_csv("maupassant_iter/MAUPASSANT_une-vie_Kraken-basetxt_lg_spacy_iterations.csv")

liste_x = []
for i in dic_x:
    for j in dic_x[i]:
        liste_x.append(j)
    
liste_y = []
for i in dic_y:
    for j in dic_y[i]:
        liste_y.append(j)

print(liste_y)

res = to_bio(liste_x,liste_y)

print(res[0],"\n\n",res[1])

[['Rouen', 'LOC'], ['Rouen', 'LOC'], ['Rouen', 'LOC'], ['Rouen', 'LOC'], ['Rouen', 'LOC'], ['Rouen', 'LOC'], ['Rouen', 'LOC'], ['Rouen', 'LOC'], ['Paris', 'LOC'], ['Paris', 'LOC'], ['Paris', 'LOC'], ['Paris', 'LOC'], ['Paris', 'LOC'], ['Paris', 'LOC'], ['Paris', 'LOC'], ['Paris', 'LOC'], ['Paris', 'LOC'], ['Paris', 'LOC'], ['Paris', 'LOC'], ['Yport', 'LOC'], ['Yport', 'LOC'], ['Yport', 'LOC'], ['Yport', 'LOC'], ['Yport', 'LOC'], ['Yport', 'LOC'], ['Yport', 'LOC'], ['Yport', 'LOC'], ['Yport', 'LOC'], ['Adelaide', 'LOC'], ['Adelaide', 'LOC'], ['Adelaide', 'LOC'], ['ILa', 'LOC'], ['ILa', 'LOC'], ['Normandie', 'LOC'], ['Normandie', 'LOC'], ['Normandie', 'LOC'], ['Normandie', 'LOC'], ['Normandie', 'LOC'], ['E', 'LOC'], ['E', 'LOC'], ['E', 'LOC'], ['E', 'LOC'], ['E', 'LOC'], ['E', 'LOC'], ['E', 'LOC'], ['E', 'LOC'], ['E', 'LOC'], ['E', 'LOC'], ['E', 'LOC'], ['E', 'LOC'], ['E', 'LOC'], ['E', 'LOC'], ['E', 'LOC'], ['E', 'LOC'], ['E', 'LOC'], ['E', 'LOC'], ['E', 'LOC'], ['E', 'LOC'], ['E', 'LOC

In [8]:
#######TESt sur les .json#######

json_x = lire_json("aimardTra_sm/AIMARD_les-trappeurs_PP.txt_sm_spacy.json-concat.json")
json_y = lire_json("aimardTra_sm/AIMARD_les-trappeurs_Kraken-base.txt_sm_spacy.json-concat.json")

liste_x = []
for i in json_x:
    liste_x.append([i,"LOC"])
    
liste_y = []
for i in json_y:
    liste_y.append([i,"LOC"])

#print(liste_y)

res = to_bio(liste_x,liste_y)

print(res[0],"\n\n",res[1])

Prologue B-LOC
l'Amérique B-LOC
l'Amérique B-LOC
Amérique B-LOC
Espagnols B-LOC
Incas B-LOC
l'Europe B-LOC
l'Amérique B-LOC
États B-LOC
État B-LOC
rioGila B-LOC
État  B-LOC
sierraMadre B-LOC
golfedeCalifornie B-LOC
sierraMadre B-LOC
Durango B-LOC
ÉtatsdeDurango B-LOC
Guadalajara B-LOC
Pacifique B-LOC
Sonora B-LOC
rioGila B-LOC
sierraMadre B-LOC
Indiens B-LOC
Comanches B-LOC
Pawnees B-LOC
Pimas B-LOC
Opatas B-LOC
Apaches B-LOC
Sonora B-LOC
Guaymas B-LOC
Hermosillo B-LOC
Arispe B-LOC
Pitic B-LOC
Pacifique B-LOC
Cerro B-LOC
Montagnedelacloche B-LOC
 B-LOC
 I-LOC
Leperos B-LOC
Gambusinos B-LOC
Rateros B-LOC
Rosario B-LOC
rueduRosaire B-LOC
Mexique B-LOC
Indiens B-LOC
Lassez B-LOC
Eh B-LOC
Ciboto B-LOC
Rafaël B-LOC
Mexicains B-LOC
Nul B-LOC
d'Espagne B-LOC
Excellence B-LOC
Excellence B-LOC
Excellence B-LOC
II B-LOC
haciendadelMilagro B-LOC
Pérou B-LOC
haciendadelMilagro B-LOC
Cordoue B-LOC
Panama B-LOC
haciendadelMilagro B-LOC
Mexique B-LOC
rioGila B-LOC
Lesoleil B-LOC
Eusébio B-LOC
Apaches

In [4]:
adam_kraken_base = lire_fichier("corpora/ADAM/ADAM_krakenbase/ADAM_Mon-village_Kraken-base.txt")
adam_tesFra_png = lire_fichier("corpora/ADAM/ADAM_TesseractFra-PNG/ADAM_Mon-village_TesseractFra-PNG.txt")
adam_tes_png = lire_fichier("corpora/ADAM/ADAM_TESSERACT-PNG/ADAM_Mon-village_Tesserat-PNG.txt")
adam_PP = lire_fichier("corpora/ADAM/ADAM_kraken-17/ADAM_Mon-village_PP.txt")

aimardRiv_kraken_base = lire_fichier("corpora/AIMARD_RIVIERE/AIMARD-RIVIERE_kraken-base/AIMARD_belle-riviere_Kraken-base.txt")
aimardRiv_tesFra_png = lire_fichier("corpora/AIMARD_RIVIERE/AIMARD-RIVIERE_TesseractFra-PNG/AIMARD_belle-riviere_TesseractFra-PNG.txt")
aimardRiv_tes_png = lire_fichier("corpora/AIMARD_RIVIERE/AIMARD-RIVIERE_TESSERACT-PNG/AIMARD_belle-rivière_Tesseract-PNG.txt")
aimardRiv_PP = lire_fichier("corpora/AIMARD_RIVIERE/AIMARD-RIVIERE_kraken-17/AIMARD_belle-riviere_PP.txt")

aimardTra_kraken_base = lire_fichier("corpora/AIMARD_TRAPPEURS/AIMARD-TRAPPEURS_kraken-base/AIMARD_les-trappeurs_Kraken-base.txt")
aimardTra_tesFra_png = lire_fichier("corpora/AIMARD_TRAPPEURS/AIMARD-TRAPPEURS_TesseractFra-PNG/AIMARD_les-trappeurs_TesseractFra-PNG.txt")
aimardTra_tes_png = lire_fichier("corpora/AIMARD_TRAPPEURS/AIMARD-TRAPPEURS_TESSERACT-PNG/AIMARD_les-trappeurs_Tesseract-PNG.txt")
aimardTra_PP = lire_fichier("corpora/AIMARD_TRAPPEURS/AIMARD-TRAPPEURS_kraken-17/AIMARD_les-trappeurs_PP.txt")

audoux_kraken_base = lire_fichier("corpora/AUDOUX/AUDOUX_kraken-base/AUDOUX_Marie-Claire_Kraken-base.txt")
audoux_tesFra_png = lire_fichier("corpora/AUDOUX/AUDOUX_TesseractFra-PNG/AUDOUX_Marie-Claire_TesseractFra-PNG.txt")
audoux_tes_png = lire_fichier("corpora/AUDOUX/AUDOUX_TESSERACT-PNG/AUDOUX_Marie-Claire_Tesseract-PNG.txt")
audoux_PP = lire_fichier("corpora/AUDOUX/AUDOUX_kraken-17/AUDOUX_Marie-Claire_PP.txt")

balzac_kraken_base = lire_fichier("corpora/BALZAC/BALZAC_kraken-base/BALZAC_Savarus_Kraken-base.txt")
balzac_tesFra_png = lire_fichier("corpora/BALZAC/BALZAC_TesseractFra-PNG/BALZAC_Savarus_TesseractFra-PNG.txt")
balzac_tes_png = lire_fichier("corpora/BALZAC/BALZAC_TESSERACT-PNG/BALZAC_Savarus_Tesseract-PNG.txt")
balzac_PP = lire_fichier("corpora/BALZAC/BALZAC_kraken-17/BALZAC_Savarus_PP.txt")

carraud_kraken_base = lire_fichier("corpora/CARRAUD/CARRAUD_kraken-base/CARRAUD_petite-Jeanne_Kraken-base.txt")
carraud_tesFra_png = lire_fichier("corpora/CARRAUD/CARRAUD_TesseractFra-PNG/CARRAUD_petite-Jeanne_TesseractFra-PNG.txt")
carraud_tes_png = lire_fichier("corpora/CARRAUD/CARRAUD_TESSERACT-PNG/CARRAUD_petite-Jeanne_Tesseract-PNG.txt")
carraud_PP = lire_fichier("corpora/CARRAUD/CARRAUD_kraken-17/CARRAUD_petite-Jeanne_PP.txt")

dash_kraken_base = lire_fichier("corpora/DASH/DASH_kraken-base/DASH_chateau-de-Pinon-V1_Kraken-base.txt")
dash_tesFra_png = lire_fichier("corpora/DASH/DASH_TesseractFra-PNG/DASH_chateau-de-Pinon-V1_TesseractFra-PNG.txt")
dash_tes_png = lire_fichier("corpora/DASH/DASH_TESSERACT-PNG/DASH_chateau-de-Pinon-V1_Tesseract-PNG.txt")
dash_PP = lire_fichier("corpora/DASH/DASH_kraken-17/DASH_chateau-de-Pinon-V1_PP.txt")

daudet_kraken_base = lire_fichier("corpora/DAUDET/DAUDET_kraken-base/DAUDET_petit-chose_Kraken-base.txt")
daudet_tesFra_png = lire_fichier("corpora/DAUDET/DAUDET_TesseractFra-PNG/DAUDET_petit-chose_TesseractFra-PNG.txt")
daudet_tes_png = lire_fichier("corpora/DAUDET/DAUDET_TESSERACT-PNG/DAUDET_petit-chose_Tesseract-PNG.txt")
daudet_PP = lire_fichier("corpora/DAUDET/DAUDET_kraken-17/DAUDET_petit-chose_PP.txt")

flaubert_kraken_base = lire_fichier("corpora/FLAUBERT/FLAUBERT_kraken-base/FLAUBERT_education-sentimentale_Kraken-base.txt")
flaubert_tesFra_png = lire_fichier("corpora/FLAUBERT/FLAUBERT_TesseractFra-PNG/FLAUBERT_education-sentimentale_TesseractFra-PNG.txt")
flaubert_tes_png = lire_fichier("corpora/FLAUBERT/FLAUBERT_TESSERACT-PNG/FLAUBERT_education-sentimentale_Tesseract-PNG.txt")
flaubert_PP = lire_fichier("corpora/FLAUBERT/FLAUBERT_kraken-17/FLAUBERT_education-sentimentale_PP.txt")

maup_kraken_base = lire_fichier("corpora/MAUPASSANT/MAUPASSANT_kraken-base/MAUPASSANT_une-vie_Kraken-base.txt")
maup_tesFra_png = lire_fichier("corpora/MAUPASSANT/MAUPASSANT_TesseractFra-PNG/MAUPASSANT_une-vie_TesseractFra-PNG.txt")
maup_tes_png = lire_fichier("corpora/MAUPASSANT/MAUPASSANT_TESSERACT-PNG/MAUPASSANT_une-vie_Tesseract-PNG.txt")
maup_PP = lire_fichier("corpora/MAUPASSANT/MAUPASSANT_kraken-17/MAUPASSANT_une-vie_PP.txt")

noailles_kraken_base = lire_fichier("corpora/NOAILLES/NOAILLES_kraken-base/NOAILLES_la-nouvelle-esperance_Kraken-base.txt")
noailles_tesFra_png = lire_fichier("corpora/NOAILLES/NOAILLES_TesseractFra-PNG/NOAILLES_la-nouvelle-esperance_TesseractFra-PNG.txt")
noailles_tes_png = lire_fichier("corpora/NOAILLES/NOAILLES_TESSERACT-PNG/NOAILLES_la-nouvelle-esperance_Tesseract-PNG.txt")
noailles_PP = lire_fichier("corpora/NOAILLES/NOAILLES_kraken-17/NOAILLES_la-nouvelle-esperance_PP.txt")

In [29]:
#######Test sur les Textes######

txt_x = aimardRiv_PP
doc_X = nlp(txt_x)

In [26]:
txt_y = aimardRiv_tesFra_png
doc_y = nlp(txt_y)

In [31]:
liste_X = []
for ent in doc_X.ents:
    if ent.label_:
        liste_X.append([ent.text, ent.label_])

liste_y = []
for ent in doc_y.ents:
    if ent.label_:
        liste_y.append([ent.text, ent.label_])
    
res = to_bio_LOC(liste_X,liste_y)

print(res[0],"\n######################\n",res[1])

Le B-MISC
Fort I-MISC
Duquesne I-MISC
I I-MISC
comte B-PER
de I-PER
Jumonville I-PER
Louis B-PER
XIV I-PER
Louis B-PER
XV I-PER
l'Amérique B-LOC
la B-LOC
France I-LOC
Canada B-LOC
Nouvelle B-LOC
France I-LOC
Anglais B-LOC
Angleterre B-LOC
fécond B-PER
Mississipiens B-LOC
Voltaire B-PER
la B-LOC
France I-LOC
Canada B-LOC
Européens B-LOC
Indiens B-LOC
Nouvelle B-LOC
France I-LOC
baie B-LOC
d'Hudson I-LOC
golfe B-LOC
du I-LOC
Mexique I-LOC
Nouvelle B-LOC
Orléans I-LOC
la B-LOC
France I-LOC
Canada B-LOC
l'Amérique B-LOC
Ferney B-LOC
la B-LOC
France I-LOC
Amérique B-LOC
Anglais B-LOC
Français B-LOC
Cette B-MISC
guerre I-MISC
inique I-MISC
Anglais B-LOC
Ohio B-LOC
Belle B-LOC
Rivière I-LOC
Virginie B-LOC
Angleterre B-LOC
Indiens B-LOC
Royal B-ORG
Marine I-ORG
Canadiens B-LOC
Peaux B-LOC
Rouges I-LOC
Dinwidie B-ORG
Virginie B-LOC
Ohio B-LOC
Washington B-PER
Angleterre B-LOC
Ward B-PER
Ohio B-LOC
Français B-LOC
M. B-PER
de I-PER
Contrecœur I-PER
fort B-LOC
Duquesne I-LOC
Pittsbourg B-LOC
États

In [32]:
with open("test_PP.bio","w",encoding="utf-8") as f:
    f.write(res[0])

with open("test_predict.bio","w",encoding="utf-8") as f:
    f.write(res[1])

In [24]:
with open("test_predict.bio","r",encoding="utf-8") as f:
    txt = f.read()
    
print(txt)

Amérique B-LOC
du I-LOC
Sud I-LOC
décou—Verte B-LOC
l’A—mérique B-LOC
Amérique B-LOC
colo—nie B-LOC
Espagnols B-LOC
parcourez B-LOC
leaujourd’hui I-LOC
Incas B-LOC
l’heure B-LOC
s’alfaisse B-LOC
len I-LOC
États B-LOC
Mexicaine B-LOC
l’État B-LOC
de I-LOC
Sonora I-LOC
l’ontourent B-LOC
et I-LOC
de I-LOC
ses I-LOC
frottements I-LOC
continuels I-LOC
État B-LOC
l’ B-LOC
l’ B-LOC
Sierra B-LOC
Madre I-LOC
golfe B-LOC
de I-LOC
Cali I-LOC
fornie I-LOC
Sierra B-LOC
Madre I-LOC
Durango B-LOC
l’autre B-LOC
tournevers I-LOC
l’ouest I-LOC
États B-LOC
de I-LOC
Durango I-LOC
et I-LOC
de I-LOC
Guadalaæw I-LOC
a I-LOC
Paciﬁque B-LOC
la B-LOC
Sonore I-LOC
tempéré B-LOC
salubre B-LOC
l’or B-LOC
Vestes B-LOC
solitudesdu I-LOC
Bio I-LOC
Gila I-LOC
Sierra B-LOC
Madre I-LOC
Indiens B-LOC
Pimas B-LOC
Opatas B-LOC
blan—che B-LOC
Sonora B-LOC
H B-LOC
ermasz‘llo I-LOC
Arz B-LOC
spe I-LOC
s’appuie B-LOC
el B-LOC
Cerro I-LOC
Du I-LOC
reste I-LOC
l’après B-LOC
midi I-LOC
villed’Hermosillo B-LOC
Leperos B-LOC
Gambus

In [24]:
Super, en effet, nous obtenons bien des sorties au format BIO.
Dans un premier temps, il serait intéressant de générer un tableau qui comprendrait une colonne Référence, 
une colonne Kraken et une colonne Tesseract fra-png, nous permettant d'observer si l'alignement est plutôt correct. 
Puis qui nous permettrait de visualiser les étiquettes et les erreurs ou non dans l'étiquetage.


#####Exécution de nerval BIO avec l'invite de commande#####

#$ nerval -a test_PP.bio -p test_predict.bio

#Kraken-base (LOC - fichiers .csv)
| tag | predicted | matched | Precision | Recall | F1    | Support |
|-----|-----------|---------|-----------|--------|-------|---------|
| LOC | 734       | 112     | 0.153     | 0.375  | 0.217 | 299     |
| All | 734       | 112     | 0.153     | 0.375  | 0.217 | 299     |

#Tesseract-PNG (LOC)
| tag | predicted | matched | Precision | Recall | F1    | Support |
|-----|-----------|---------|-----------|--------|-------|---------|
| LOC | 647       | 207     | 0.32      | 0.564  | 0.408 | 367     |
| All | 647       | 207     | 0.32      | 0.564  | 0.408 | 367     |


#Kraken-base (Tout)
| tag  | predicted | matched | Precision | Recall | F1    | Support |
|------|-----------|---------|-----------|--------|-------|---------|
| PER  | 2398      | 472     | 0.197     | 0.469  | 0.277 | 1006    |
| ORG  | 156       | 2       | 0.013     | 0.25   | 0.024 | 8       |
| MISC | 493       | 45      | 0.091     | 0.188  | 0.123 | 240     |
| LOC  | 1166      | 156     | 0.134     | 0.425  | 0.204 | 367     |
| All  | 4213      | 675     | 0.16      | 0.416  | 0.231 | 1621    |

#Tesseract-PNG (TOUT)
# (error)

#caractères problématiques: ﬁ ﬂ “ ” ç Ç § ï Ï  ’́ (et d'autres que je n'ai pas pu identifier) parfrois: à 
#analyser les sorties annotées


##################################################################################
#Demo
#
#$ nerval -a demo_nerval/demo_annot.bio -p demo_nerval/demo_predict.bio
#
#
#We also provide two annotation and prediction toy files, which are identical for now and produce perfect scores. Feel free to play with the the text and entity tags in the prediction file to see the impact on the score.
#
#$ nerval -a demo/toy_test_annot.bio -p demo/toy_test_predict.bio
#
#
#You can also indicate a folder and a csv file to have multiple evaluation at once.
#
#$ nerval -c demo/mapping_file.csv -f demo/bio_folder
#
#
#And with the verbose option that's triggered by -v
#
#$ nerval -a demo/demo_annot.bio -p demo/demo_predict.bio -v

SyntaxError: invalid syntax (<ipython-input-24-d9b33737d2aa>, line 1)

In [22]:
# REGROUPEMENT DE LIGNES

test = BIO_regroup("../fichiers_bio_alignement/aimardTrappeurs_lg/aimardTrappeurs_kraken_lg.txt")
print(test)

l B-LOC B-LOC
' I-LOC I-LOC
A I-LOC I-LOC
m I-LOC I-LOC
é I-LOC I-LOC
r I-LOC I-LOC
i I-LOC I-LOC
q I-LOC I-LOC
u I-LOC I-LOC
e I-LOC I-LOC
  I-LOC O
d I-LOC B-LOC
u I-LOC I-LOC
  I-LOC I-LOC
S I-LOC I-LOC
u I-LOC O
d I-LOC B-LOC
  O I-LOC
l B-LOC I-LOC
' I-LOC I-LOC
A I-LOC I-LOC
m I-LOC I-LOC
é I-LOC I-LOC
r I-LOC I-LOC
i I-LOC I-LOC
q I-LOC I-LOC
u I-LOC I-LOC
e I-LOC I-LOC
  O O
A B-LOC B-LOC
m I-LOC I-LOC
é I-LOC I-LOC
r I-LOC I-LOC
i I-LOC I-LOC
q I-LOC I-LOC
u I-LOC I-LOC
e I-LOC I-LOC
  O O
E B-LOC B-LOC
s I-LOC I-LOC
p I-LOC I-LOC
a I-LOC I-LOC
g I-LOC I-LOC
n I-LOC I-LOC
o I-LOC I-LOC
l I-LOC I-LOC
s I-LOC I-LOC
  O O
p B-LOC B-LOC
a I-LOC I-LOC
r I-LOC I-LOC
c I-LOC I-LOC
o I-LOC I-LOC
u I-LOC I-LOC
r I-LOC I-LOC
e I-LOC I-LOC
z I-LOC I-LOC
  I-LOC I-LOC
l I-LOC I-LOC
e I-LOC I-LOC
  O O
I B-LOC B-LOC
n I-LOC I-LOC
c I-LOC I-LOC
a I-LOC I-LOC
s I-LOC I-LOC
  O O
l B-LOC I-LOC
' I-LOC I-LOC
E I-LOC I-LOC
u I-LOC I-LOC
r I-LOC I-LOC
o I-LOC I-LOC
p I-LOC I-LOC
e I-LOC I-LOC
  